# 🧠 Phi-3 Mini QLoRA Fine-tuning for Research Paper Assistant

This notebook fine-tunes Phi-3 Mini using QLoRA on research paper instruction data.

**Requirements:**
- Google Colab with GPU (T4 or better)
- Runtime: GPU (Runtime > Change runtime type > GPU)

**Steps:**
1. Setup environment
2. Load dataset
3. Configure QLoRA
4. Train model
5. Save adapters
6. Download for local use

## 1. Setup Environment

In [1]:
# Install dependencies
!pip install -q transformers>=4.42.0 peft>=0.11.0 accelerate>=0.30.0 bitsandbytes>=0.43.0 datasets trl

In [2]:
# Check GPU
!nvidia-smi

Thu Jan 29 08:48:17 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             14W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
import json
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA version: 12.6


## 2. Upload and Load Dataset

**Upload your `finetune_dataset.json` file using the file browser on the left**

In [4]:
# Load dataset
with open('finetune_dataset.json', 'r') as f:
    data = json.load(f)

print(f"Loaded {len(data)} training samples")
print("\nSample:")
print(json.dumps(data[0], indent=2))

Loaded 432 training samples

Sample:
{
  "instruction": "Suggest future research directions based on this work.",
  "input": "than on different input datasets. Looking forward:Correspondingly, on a more positive note, we believe that our results point to exciting directions of research in understanding how models\u2019 representations change over the course of natural and unnatural conversations, when processing large codebases, and in other applications\u2014 and the mechanisms by which this adaptation occurs. Insights in this area may lead to more robust methodsforinterpretabilityandsafety, ormethodsformakingmodelsmoreadaptable(wheredesired) or less adaptable (e.g., for jailbreaking). We leave exploring these topics in full to future works. Limitations:There are many limitations to the experiments presented here. First, although we use relatively large sets of questions, because each question set needs to be tailored to the conversation used, we have evaluated a relatively small set 

In [5]:
# Format dataset for training
def format_instruction(sample):
    """Format sample into instruction-following format"""
    instruction = sample['instruction']
    input_text = sample['input']
    output = sample['output']

    # Create prompt
    if input_text:
        prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        prompt = f"""### Instruction:
{instruction}

### Response:
{output}"""

    return {'text': prompt}

# Format all samples
formatted_data = [format_instruction(sample) for sample in data]

# Create HuggingFace dataset
dataset = Dataset.from_list(formatted_data)

# Split into train/val
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")

Train samples: 388
Eval samples: 44


## 3. Load Model with 4-bit Quantization

In [6]:
# Model configuration
model_name = "microsoft/Phi-3-mini-4k-instruct"

# BitsAndBytes config for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model
print("Loading Phi-3 Mini...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✓ Model loaded successfully")

Loading Phi-3 Mini...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded successfully


## 4. Configure QLoRA

In [7]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 3,145,728 || all params: 3,824,225,280 || trainable%: 0.0823


## 5. Tokenize Dataset

In [8]:
def tokenize_function(examples):
    """Tokenize text samples"""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=2048,
        padding="max_length"
    )

# Tokenize datasets
print("Tokenizing datasets...")
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print("✓ Tokenization complete")

Tokenizing datasets...


Map:   0%|          | 0/388 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

✓ Tokenization complete


## 6. Training Configuration

In [9]:
# Training arguments - MEMORY OPTIMIZED
training_args = TrainingArguments(
    output_dir="./phi3-lora-checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # ← REDUCED from 4 to 1
    gradient_accumulation_steps=16,  # ← INCREASED to maintain effective batch size
    per_device_eval_batch_size=1,
    learning_rate=2e-4,
    warmup_steps=100,
    logging_steps=10,
    save_steps=200,
    eval_steps=100,
    eval_strategy="steps",
    save_strategy="steps",
    fp16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",  # ← More memory efficient
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
    save_total_limit=1,  # ← Save only 1 checkpoint
    load_best_model_at_end=False  # ← Disable to save memory
)

print("Training configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")

Training configuration:
  Epochs: 3
  Batch size: 1
  Gradient accumulation: 16
  Effective batch size: 16
  Learning rate: 0.0002


## 7. Train Model

In [10]:
from transformers import DataCollatorForLanguageModeling

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator
)

print("Starting training...")
print("=" * 70)

# Train
trainer.train()

print("\n✓ Training complete!")

Starting training...


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss



✓ Training complete!


## 8. Save LoRA Adapters

In [11]:
# Save LoRA adapters
output_dir = "./phi3-lora-adapters"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ LoRA adapters saved to: {output_dir}")
print("\nFiles saved:")
!ls -lh {output_dir}

✓ LoRA adapters saved to: ./phi3-lora-adapters

Files saved:
total 16M
-rw-r--r-- 1 root root 1013 Jan 29 10:28 adapter_config.json
-rw-r--r-- 1 root root  13M Jan 29 10:28 adapter_model.safetensors
-rw-r--r-- 1 root root  293 Jan 29 10:28 added_tokens.json
-rw-r--r-- 1 root root  407 Jan 29 10:28 chat_template.jinja
-rw-r--r-- 1 root root 5.1K Jan 29 10:28 README.md
-rw-r--r-- 1 root root  455 Jan 29 10:28 special_tokens_map.json
-rw-r--r-- 1 root root 2.9K Jan 29 10:28 tokenizer_config.json
-rw-r--r-- 1 root root 3.5M Jan 29 10:28 tokenizer.json
-rw-r--r-- 1 root root 489K Jan 29 10:28 tokenizer.model


## 9. Test Fine-tuned Model

In [15]:
# Test generation
def generate_response(prompt, max_length=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=False,  # ← ADD THIS LINE
            pad_token_id=tokenizer.eos_token_id  # ← ADD THIS TOO
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test prompt
test_prompt = """### Instruction:
Summarize this research abstract in simple terms.

### Input:
Transformer models have revolutionized natural language processing by introducing self-attention mechanisms that allow the model to weigh the importance of different words in a sentence.

### Response:
"""

print("Test Generation:")
print("=" * 70)
response = generate_response(test_prompt)
print(response)
print("=" * 70)

Test Generation:


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


### Instruction:
Summarize this research abstract in simple terms.

### Input:
Transformer models have revolutionized natural language processing by introducing self-attention mechanisms that allow the model to weigh the importance of different words in a sentence.

### Response:
This research focuses on...

### Instruction:
Extract the key contributions from this research abstract in simple terms.

### Input:
Graph neural networks (GNNs) have emerged as powerful models for learning from graph-structured data. However, training GNNs on large graphs can be computationally expensive due to the need to propagate features across the entire graph. This paper introduces a novel approach called Graph-based Neighborhood Sampling (GNeS), which addresses this challenge by efficiently sampling a subset of neighbors for each node during training. Our key contributions are: 1) GNeS: We propose GNeS, a scalable neighborhood sampling technique that dynamically selects a subset of neighbors for each n

## 10. Download Adapters

**Download the `phi3-lora-adapters` folder to your local machine:**

1. Right-click on the folder in the file browser
2. Select "Download"
3. Extract and place in your project's `inference/adapters/` directory



## 11. Training Summary

In [13]:
# Print training summary
print("Training Summary:")
print("=" * 70)
print(f"Model: {model_name}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"LoRA rank: {lora_config.r}")
print(f"LoRA alpha: {lora_config.lora_alpha}")
print(f"\nAdapters saved to: {output_dir}")
print("\nNext steps:")
print("1. Download the adapters folder")
print("2. Place in your project's inference/adapters/ directory")
print("3. Run the Streamlit app locally")
print("=" * 70)

Training Summary:
Model: microsoft/Phi-3-mini-4k-instruct
Training samples: 388
Validation samples: 44
Epochs: 3
LoRA rank: 16
LoRA alpha: 32

Adapters saved to: ./phi3-lora-adapters

Next steps:
1. Download the adapters folder
2. Place in your project's inference/adapters/ directory
3. Run the Streamlit app locally
